In [5]:
import pandas as pd

In [3]:
flights = pd.read_parquet("../resources/flights_clean.parquet")
flights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Mes,Semana,DiaSemana,Hora,HourDuration
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,10,42,4,11,2.750000
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,7,27,1,0,4.000000
2,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,6,26,1,15,0.666667
3,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,2,6,5,8,1.166667
4,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0,9,36,4,17,0.916667


In [5]:
import json

data = {
    "aeronaves": flights["Aeronave"]
    .drop_duplicates()
    .sort_values(ascending=True)
    .values.tolist(),
    "tipo_lugares": {
        aeronave: dest_type
        for aeronave, dest_type in flights[["ArrivalStation", "Destination_Type"]]
        .drop_duplicates()
        .sort_values(["ArrivalStation", "Destination_Type"])
        .to_records(index=False)
    },
    "origenes": flights["DepartureStation"]
    .drop_duplicates()
    .sort_values()
    .values.tolist(),
    "destinos": flights["ArrivalStation"]
    .drop_duplicates()
    .sort_values()
    .values.tolist(),
}

# with open("../src/data/vuelos.json", "w") as f:
#     json.dump(data, f)

In [6]:
sales = pd.read_parquet("../resources/sales_clean.parquet")
sales.head()

,index,Flight_ID,ProductType,ProductName,Quantity,TotalSales
0,0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0
1,1,08f6f97437df8db101b050f1110be656,Licores,Jw Red Label,2,240.0
2,2,4e09c949826a77207868412baeff6d30,Licores,Jack And Coke,8,576.0
3,3,c3e1568fe46c68d3174681d322d412b3,Licores,Jw Red Label,2,240.0
4,5,34ae909bf3699372fcd66f02af0bdd54,Licores,Ron Bacardi,2,240.0


In [7]:
def parse_model_name(product: str):
    """
    index se refiere al índice de `product_dict_inv`
    """
    return "_".join(product.lower().replace("/", "_").split())

In [8]:
sales[sales["ProductName"].str.contains("agua", case=False)]

,index,Flight_ID,ProductType,ProductName,Quantity,TotalSales
975085,1224385,44422056eeee03c23dd7b6b383124f94,Refrescos,Agua Natural 600 Ml,4,180.0
975086,1224386,4875022d921228a2fc443d0df047899a,Refrescos,Agua Natural 600 Ml,21,945.0
975087,1224387,495bad4cded089e034b2ff6d04109931,Refrescos,Agua Natural 600 Ml,6,270.0
975088,1224388,fec8ef7ed732deaccc54fe9b8fc665b9,Refrescos,Agua Natural 600 Ml,6,270.0
975089,1224389,647b09d0823ffb6639a50f7d38827855,Refrescos,Agua Natural 600 Ml,2,96.0
...,...,...,...,...,...,...
2057103,2509959,86996c0fc5ba1b87a2f3b1aade716e6d,Refrescos,Agua Natural 600 Ml,1,45.0
2057104,2509960,e521f26b26b1ecacc56f7f8bd2ee5792,Refrescos,Agua Natural 600 Ml,1,48.0
2057105,2509961,3523e5db128cebc5b0f786d4380eb443,Refrescos,Agua Natural 600 Ml,1,48.0
2057106,2509962,1c2bed223a3c73c4302889b2275a3a77,Refrescos,Agua Natural 600 Ml,1,45.0


In [9]:
sales["ProductName"].drop_duplicates().replace(
    {"Sabritas Originales": "sabritas", "Agua Natural 600 Ml": "agua"}
).apply(parse_model_name).sort_values().values

array(['agua', 'amstel_ultra', 'arandano', 'arandano_mango_mix',
       'arcoiris', 'baileys', 'baileys', 'cafe_19_cafe_clasico',
       'cafe_19_capuchino', 'cafe_19_chiapas', 'cafe_costa',
       'cafe_de_olla', 'carne_seca_habanero', 'carne_seca_original',
       'cerveza_charter', 'cheetos', 'cheetos_flamin_hot', 'chokis',
       'ciel_mineralizada', 'coca_cola_dieta', 'coca_cola_regular',
       'coca_sin_azucar', 'combo_stl', 'corajillo', 'corajillo_baileys',
       'cuerno_clasico_de_pavo', 'dip_de_queso', 'doritos_nacho',
       'eco_holder', 'emperador_chocolate', 'emperador_vainilla',
       'fanta_de_naranja', 'fritos_limon_y_sal',
       'frutos_secos_enchilados',
       'galleta_de_arandano_relleno_de_q_crema',
       'galleta_de_chispas_de_chocolate', 'galleta_de_chocolate',
       'go_nuts', 'gomita_enchilada_la_cueva', 'heineken_0',
       'heineken_original', 'heineken_silver', 'jack_and_coke',
       'jugo_de_mango', 'jugo_de_manzana', 'jw_red_label', 'jw_red_label',


In [10]:
sales["ProductType"].drop_duplicates().sort_values().values.tolist()

['Bebidas Calientes',
 'Botanas',
 'Galletas',
 'Lacteos',
 'Licores',
 'Perecederos',
 'Refrescos',
 'Sopas']

In [11]:
product_index = {
    0: "Carne Seca Habanero",
    1: "Jw Red Label ",
    2: "Jack And Coke",
    3: "Jw Red Label",
    4: "Ron Bacardi",
    5: "Baileys",
    6: "Corajillo",
    7: "Muffin Integral",
    8: "Tequila 7 Leguas Reposado",
    9: "Arandano Mango Mix",
    10: "Quaker Granola",
    11: "Tequila 7 Leguas Blanco",
    12: "Sol Clamato",
    13: "Quaker Avena Frutos Rojos",
    14: "Go Nuts",
    15: "Arandano",
    16: "Nutty Berry Mix",
    17: "Frutos Secos Enchilados",
    18: "Te Relax",
    19: "Dip De Queso",
    20: "Te Frutos Rojos",
    21: "Ultra Seltzer Frambuesa",
    22: "Corajillo Baileys ",
    23: "Vino Tinto Cria Cuervos",
    24: "Te Manzanilla Jengibre",
    25: "Baileys ",
    26: "Nueces De Arbol Mix",
    27: "Carne Seca Original",
    28: "Luxury Nut Mix",
    29: "Galleta De Chispas De Chocolate",
    30: "Tostitos Nachos Con Dip",
    31: "Protein Adventure",
    32: "Vino Blanco Cria Cuervos ",
    33: "Galleta De Chocolate",
    34: "Topochico Seltzer Fresa-Guayaba",
    35: "Te Vainilla",
    36: "Cafe De Olla",
    37: "Galleta De Arandano Relleno De Q/Crema",
    38: "Cafe 19 Cafe Clasico",
    39: "Licor Charter",
    40: "Quaker Avena Moras",
    41: "Quaker Natural Balance",
    42: "Combo Stl",
    43: "Eco Holder",
    44: "Cerveza Charter",
    45: "Maxi Combo",
    46: "Chokis",
    47: "Sprite",
    48: "Cheetos",
    49: "Arcoiris",
    50: "Tostitos",
    51: "Xx Lager",
    52: "Xx Ultra",
    53: "Cafe Costa",
    54: "Nissin Res",
    55: "Rancheritos",
    56: "Amstel Ultra",
    57: "Nissin Fuego",
    58: "Tecate Light",
    59: "Doritos Nacho",
    60: "Jugo De Mango",
    61: "Mafer Sin Sal",
    62: "Ruffles Queso",
    63: "Sidral Mundet",
    64: "Nissin Picante",
    65: "Panini Clasico",
    66: "Cafe 19 Chiapas",
    67: "Coca Cola Dieta",
    68: "Coca Sin Azucar",
    69: "Heineken Silver",
    70: "Jugo De Manzana",
    71: "Panini Integral",
    72: "Fanta De Naranja",
    73: "Nishikawa Salado",
    74: "Cafe 19 Capuchino",
    75: "Ciel Mineralizada",
    76: "Coca Cola Regular",
    77: "Heineken Original",
    78: "Nissin Limon Y Habanero",
    79: "Heineken 0",
    80: "Gomita Enchilada La Cueva",
    81: "Kacang Flaming Hot",
    82: "Leche De Fresa Sc",
    83: "Nishikawa Japones",
    84: "Cheetos Flamin Hot",
    85: "Emperador Vainilla",
    86: "Fritos Limon Y Sal",
    87: "Nissin Dark Dragon",
    88: "Agua Natural 600 Ml",
    89: "Emperador Chocolate",
    90: "Mega Cuerno Clasico",
    91: "Sabritas Flamin Hot",
    92: "Sabritas Originales",
    93: "Leche De Chocolate Sc",
    94: "Cuerno Clasico De Pavo",
    95: "Topochico Seltzer Mango",
    96: "Vino Tinto Sangre De Toro",
}

product_name_index = {v: k for k, v in product_index.items()}
product_name_index

{'Carne Seca Habanero': 0,
 'Jw Red Label ': 1,
 'Jack And Coke': 2,
 'Jw Red Label': 3,
 'Ron Bacardi': 4,
 'Baileys': 5,
 'Corajillo': 6,
 'Muffin Integral': 7,
 'Tequila 7 Leguas Reposado': 8,
 'Arandano Mango Mix': 9,
 'Quaker Granola': 10,
 'Tequila 7 Leguas Blanco': 11,
 'Sol Clamato': 12,
 'Quaker Avena Frutos Rojos': 13,
 'Go Nuts': 14,
 'Arandano': 15,
 'Nutty Berry Mix': 16,
 'Frutos Secos Enchilados': 17,
 'Te Relax': 18,
 'Dip De Queso': 19,
 'Te Frutos Rojos': 20,
 'Ultra Seltzer Frambuesa': 21,
 'Corajillo Baileys ': 22,
 'Vino Tinto Cria Cuervos': 23,
 'Te Manzanilla Jengibre': 24,
 'Baileys ': 25,
 'Nueces De Arbol Mix': 26,
 'Carne Seca Original': 27,
 'Luxury Nut Mix': 28,
 'Galleta De Chispas De Chocolate': 29,
 'Tostitos Nachos Con Dip': 30,
 'Protein Adventure': 31,
 'Vino Blanco Cria Cuervos ': 32,
 'Galleta De Chocolate': 33,
 'Topochico Seltzer Fresa-Guayaba': 34,
 'Te Vainilla': 35,
 'Cafe De Olla': 36,
 'Galleta De Arandano Relleno De Q/Crema': 37,
 'Cafe 19 C

In [12]:
{
    product: i
    for i, product in enumerate(
        sales["ProductName"]
        .drop_duplicates()
        .replace({"Sabritas Originales": "sabritas", "Agua Natural 600 Ml": "agua"})
        .apply(parse_model_name)
        .sort_values()
        .values.tolist()
    )
}

{'agua': 0,
 'amstel_ultra': 1,
 'arandano': 2,
 'arandano_mango_mix': 3,
 'arcoiris': 4,
 'baileys': 6,
 'cafe_19_cafe_clasico': 7,
 'cafe_19_capuchino': 8,
 'cafe_19_chiapas': 9,
 'cafe_costa': 10,
 'cafe_de_olla': 11,
 'carne_seca_habanero': 12,
 'carne_seca_original': 13,
 'cerveza_charter': 14,
 'cheetos': 15,
 'cheetos_flamin_hot': 16,
 'chokis': 17,
 'ciel_mineralizada': 18,
 'coca_cola_dieta': 19,
 'coca_cola_regular': 20,
 'coca_sin_azucar': 21,
 'combo_stl': 22,
 'corajillo': 23,
 'corajillo_baileys': 24,
 'cuerno_clasico_de_pavo': 25,
 'dip_de_queso': 26,
 'doritos_nacho': 27,
 'eco_holder': 28,
 'emperador_chocolate': 29,
 'emperador_vainilla': 30,
 'fanta_de_naranja': 31,
 'fritos_limon_y_sal': 32,
 'frutos_secos_enchilados': 33,
 'galleta_de_arandano_relleno_de_q_crema': 34,
 'galleta_de_chispas_de_chocolate': 35,
 'galleta_de_chocolate': 36,
 'go_nuts': 37,
 'gomita_enchilada_la_cueva': 38,
 'heineken_0': 39,
 'heineken_original': 40,
 'heineken_silver': 41,
 'jack_and_c

In [16]:
import json

data = {
    "productos": (
        sales["ProductName"]
        .drop_duplicates()
        .replace({"Sabritas Originales": "sabritas", "Agua Natural 600 Ml": "agua"})
        .apply(parse_model_name)
        .sort_values()
        .values.tolist()
    ),
    "product_index": {
        product: i
        for i, product in enumerate(
            sales["ProductName"]
            .drop_duplicates()
            .replace({"Sabritas Originales": "sabritas", "Agua Natural 600 Ml": "agua"})
            .apply(parse_model_name)
            .sort_values()
            .values.tolist()
        )
    },
    "categorias": {
        product: type_
        for product, type_ in sales[["ProductName", "ProductType"]]
        .drop_duplicates()
        .sort_values("ProductType")
        .to_records(index=False)
    },
    "categorias_unicas": sales["ProductType"]
    .drop_duplicates()
    .sort_values()
    .values.tolist(),
    "productos_nombres": {
        product: product_name
        for product, product_name in zip(
            (
                sales["ProductName"]
                .drop_duplicates()
                .replace(
                    {"Sabritas Originales": "sabritas", "Agua Natural 600 Ml": "agua"}
                )
                .apply(parse_model_name)
                .sort_values()
                .values.tolist()
            ),
            sales["ProductName"].drop_duplicates().sort_values().values.tolist(),
        )
    },
}

with open("../src/data/productos.json", "w") as f:
    json.dump(data, f)

In [8]:
data["tipo_lugares"]

{'AB': 'Playa',
 'AC': 'MX Amigos y Familia',
 'AD': 'MX Amigos y Familia',
 'AE': 'MX Amigos y Familia',
 'AF': 'Ciudad Fronteriza',
 'AI': 'Playa',
 'AJ': 'MX Amigos y Familia',
 'AK': 'Playa',
 'AL': 'MX Amigos y Familia',
 'AM': 'Playa',
 'AO': 'Ciudad Principal',
 'AP': 'MX Amigos y Familia',
 'AQ': 'Playa',
 'AR': 'Playa',
 'AS': 'MX Amigos y Familia',
 'AT': 'Ciudad Principal',
 'AU': 'Ecoturismo',
 'AV': 'MX Amigos y Familia',
 'AW': 'Ciudad Principal',
 'AX': 'MX Amigos y Familia',
 'AY': 'Playa',
 'AZ': 'Ciudad Fronteriza',
 'BA': 'Ciudad Principal',
 'BB': 'Ecoturismo',
 'BC': 'MX Amigos y Familia',
 'BD': 'Playa',
 'BE': 'Playa',
 'BF': 'MX Amigos y Familia',
 'BG': 'MX Amigos y Familia',
 'BH': 'Playa',
 'BI': 'MX Amigos y Familia',
 'BJ': 'MX Amigos y Familia',
 'BK': 'MX Amigos y Familia',
 'BL': 'Ecoturismo',
 'BM': 'Ciudad Fronteriza',
 'BN': 'MX Amigos y Familia',
 'BO': 'MX Amigos y Familia',
 'BP': 'MX Amigos y Familia',
 'BQ': 'MX Amigos y Familia',
 'BS': 'Playa',

In [30]:
flights["Capacity"].min(), flights["Capacity"].max()

(178, 240)

In [9]:
flights.loc[flights["ArrivalStation"] == "BT", "Destination_Type"].value_counts()

Destination_Type
Playa    92
Name: count, dtype: int64

In [17]:
flights["ArrivalStation"].value_counts().shape[0]

41

In [41]:
{
    aeronave: dest_type
    for aeronave, dest_type in flights[["ArrivalStation", "Destination_Type"]]
    .drop_duplicates()
    .sort_values(["ArrivalStation", "Destination_Type"])
    .to_records(index=False)
}

{'AB': 'Playa',
 'AC': 'MX Amigos y Familia',
 'AD': 'MX Amigos y Familia',
 'AE': 'MX Amigos y Familia',
 'AF': 'Ciudad Fronteriza',
 'AI': 'Playa',
 'AJ': 'MX Amigos y Familia',
 'AK': 'Playa',
 'AL': 'MX Amigos y Familia',
 'AM': 'Playa',
 'AO': 'Ciudad Principal',
 'AP': 'MX Amigos y Familia',
 'AQ': 'Playa',
 'AR': 'Playa',
 'AS': 'MX Amigos y Familia',
 'AT': 'Ciudad Principal',
 'AU': 'Ecoturismo',
 'AV': 'MX Amigos y Familia',
 'AW': 'Ciudad Principal',
 'AX': 'MX Amigos y Familia',
 'AY': 'Playa',
 'AZ': 'Ciudad Fronteriza',
 'BA': 'Ciudad Principal',
 'BB': 'Ecoturismo',
 'BC': 'MX Amigos y Familia',
 'BD': 'Playa',
 'BE': 'Playa',
 'BF': 'MX Amigos y Familia',
 'BG': 'MX Amigos y Familia',
 'BH': 'Playa',
 'BI': 'MX Amigos y Familia',
 'BJ': 'MX Amigos y Familia',
 'BK': 'MX Amigos y Familia',
 'BL': 'Ecoturismo',
 'BM': 'Ciudad Fronteriza',
 'BN': 'MX Amigos y Familia',
 'BO': 'MX Amigos y Familia',
 'BP': 'MX Amigos y Familia',
 'BQ': 'MX Amigos y Familia',
 'BS': 'Playa',